In [2]:
import pandas as pd

#filepath variable
csv_loc = "Desktop/MSU_Data_bootcamp/pandas_challenge/pymoli/purchase_data.csv"

#read csv and create data frame
purch_df = pd.read_csv(csv_loc)

#show the dataframe as it currently is
purch_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#variable players should contain the length of the column of user names, with only unique values
#this gives the total number of players

players = len(pd.unique(purch_df["SN"]))

print("The numeber of unique players who have made a purchase is: " + str(players))

The numeber of unique players who have made a purchase is: 576


In [4]:
#number of unique items
uniq_it = purch_df[["Item Name", "Price"]].copy()                   #dataframe for just items and prices .copy() is added  
                                                                    #to clear up the SettingWithCopyWarning
uniq_it.drop_duplicates(subset = "Item Name", inplace = True)       #This operation drops duplicate sales

uniq_it_num = len(uniq_it)

#average price
price = round(uniq_it["Price"].mean(), 2)                           # .mean() and .sum() are used to compute properties of
                                                                    # columns in DataFrame["column"] 
pr_disp = "$" + str(price)

#total revenue
tot_rev = purch_df["Price"].sum()

rev_d = "$" + str(tot_rev)

#number of purchase
purch_no = len(purch_df)

#basic stats dataframe: dataframe is created from a dictionary of dict = {"column": [val1, val2, ... valn]}

bsc_stat = {"Unique Items": [uniq_it_num], "Average Price": [pr_disp], "Number of Purchases": [purch_no], "Total Revenue": [rev_d]}

bsc_stat_df = pd.DataFrame(bsc_stat, columns =["Unique Items", "Average Price", "Number of Purchases", "Total Revenue"])

bsc_stat_df




,Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.04,780,$2379.77


In [8]:
#gender demographics
uniq_users = purch_df[["SN", "Age", "Gender"]].copy()         #copy data from purch_df into new dataframe
uniq_users.drop_duplicates(subset = "SN", inplace = True)     # drop duplicates to get unique user data

g_count = uniq_users["Gender"].value_counts()                 # g_count is a series of indexed counts of genders

m_perc = str(round((g_count[0]/players)*100, 2)) + "%"   #these are calculations of the percents. 

f_perc = str(round((g_count[1]/players)*100, 2)) + "%" # I might try to simplify this with a loop or function

o_perc = str(round((g_count[2]/players)*100, 2)) + "%"        # because it seems like there is redundancy here


gndr_stats = {
                "Gender": ["Male", "Female", "Other/Non-Disclosed"], 
                "Count": [g_count[0], g_count[1], g_count[2]], 
                "Percent of Players": [m_perc, f_perc, o_perc]}

gndr_df = pd.DataFrame(gndr_stats, columns = ["Gender", "Count", "Percent of Players"])

gndr_df

,Gender,Count,Percent of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other/Non-Disclosed,11,1.91%


In [15]:
#groupby gender for the original dataframe

#first I'm making a copy dataframe with the revenue data 
gen_purch_df = purch_df[["Gender", "Price"]].copy()

#group by gender and display average price per gender per purchase
ave_pur_price = gen_purch_df.groupby("Gender").mean()

ave_pur_price = ave_pur_price.rename(columns = {"Price":"Ave Purchase Price"})

#group by gender and display the sum of purchases per gender
gen_tot_rev = gen_purch_df.groupby("Gender").sum()

#groupby gender and count the purchases for each gender
gen_purch_ct = gen_purch_df.groupby("Gender").count()

gen_purch_ct = gen_purch_ct.rename(columns = {"Price":"Purchase Count"})

#these are variables to hold individual revenue totals by gender category

mtr = gen_tot_rev.loc["Male", "Price"]   

ftr = gen_tot_rev.loc["Female", "Price"]

otr = gen_tot_rev.loc["Other / Non-Disclosed", "Price"]

#this function is to format dollar amounts

def price(n):
    return "$" + str(round(n ,2))

#average spending for a member of each gender category

av_m_sp = price(mtr/g_count[0])

av_f_sp = price(ftr/g_count[1])

av_o_sp = price(otr/g_count[2])

#make dataframe for above calculations

av_sp = {"Gender": [ "Female", "Male", "Other / Non-Disclosed"],
        "Ave Total Purchase": [av_f_sp, av_m_sp, av_o_sp]}

av_sp_df = pd.DataFrame(av_sp, columns = ["Gender", "Ave Total Purchase"])

#rename the price column for the total revenue calculation
gen_tot_rev = gen_tot_rev.rename(columns = {"Price":"Total Revenue"})


#merging dataframes created by groupby operations into one dataframe
#this required 3 steps to do - it doesn't seem like you can merge multiple frames at once

m1 = pd.merge(gen_purch_ct, av_sp_df, on = "Gender")

m2 = pd.merge(ave_pur_price, gen_tot_rev, on = "Gender")

gender_stats = pd.merge(m1, m2, on = "Gender")

gender_stats



,Gender,Purchase Count,Ave Total Purchase,Ave Purchase Price,Total Revenue
0,Female,113,$4.47,3.203009,361.94
1,Male,652,$4.07,3.017853,1967.64
2,Other / Non-Disclosed,15,$4.56,3.346000,50.19
